# Generate your own GLO-30 HAND tile

This notebook will recreate a GLO-30 HAND tile, or let you generate a new one with using a different accumulation value.

## 0. Install dependencies

To run this notebook, you'll need the [HydroSAR](https://pypi.org/project/hydrosar/) Python package, the [ASF Tools](https://hyp3-docs.asf.alaska.edu/tools/asf_tools/) Python package, [numpy](https://numpy.org/), [rasterio](https://rasterio.readthedocs.io/), and [shapely](https://shapely.readthedocs.io/en/stable/manual.html). You can install them into a conda environment with:

In [ ]:
%conda install -c conda-forge asf_tools>=0.6 numpy rasterio shapely

In [ ]:
!python -m pip install hydrosar

## 1. Generate HAND
First, select a DEM tile and then provide the [virtual path](https://gdal.org/user/virtual_file_systems.html) to it

In [ ]:
dem_tile = '/vsicurl/https://copernicus-dem-30m.s3.amazonaws.com/Copernicus_DSM_COG_10_N41_00_W088_00_DEM/Copernicus_DSM_COG_10_N41_00_W088_00_DEM.tif'

Extract some metadata from the DEM tile

In [ ]:
import rasterio

with rasterio.open(dem_tile) as dem:
        dem_bounds = dem.bounds
        dem_meta = dem.meta

Buffer DEM by 0.5 degrees to avoid clipping hydrobasin boundaries

In [ ]:
from shapely.geometry import GeometryCollection, box

dem_geometry = box(*dem_bounds)
dem_buffered = GeometryCollection([dem_geometry.buffer(0.5)])

Build a temporary VRT of all DEM tiles that intersect our buffered geometry

In [ ]:
from pathlib import Path

from asf_tools.dem import prepare_dem_vrt

buffered_dem_vrt = Path(dem_tile).name.replace('.tif', '_buffered.vrt')
prepare_dem_vrt(buffered_dem_vrt, dem_buffered)

Calculate an initial HAND over the entire buffered area using the specified accumulation threshold

*Note: this will take a while.*

In [ ]:
from hydrosar.hand.calculate import calculate_hand_for_basins

accumulation_threshold = 100

hand_raster_buffered = Path(dem_tile).name.replace('DEM.tif', f'HAND_ACC{accumulation_threshold}_buffered.tif')
calculate_hand_for_basins(hand_raster_buffered, dem_buffered, buffered_dem_vrt, acc_thresh=accumulation_threshold)

Crop out buffered HAND to the original DEM tile size

In [ ]:
with rasterio.open(hand_raster_buffered) as sds:
    window = rasterio.windows.from_bounds(*dem_bounds, sds.transform)
    out_pixels = sds.read(
        1, window=window, out_shape=(dem_meta['height'], dem_meta['width']),
        resampling=rasterio.enums.Resampling.bilinear
    )

Mask the ocean pixels as identified in the WBM auxiliary DEM files

In [ ]:
import numpy as np

wmb_tile = dem_tile.replace('DEM/Copernicus', 'DEM/AUXFILES/Copernicus')
wmb_tile = wmb_tile.replace('_DEM.tif', f'_WBM.tif')

with rasterio.open(wmb_tile) as wbm:
    wbm_pixels = wbm.read(1)

out_pixels = np.ma.masked_where(wbm_pixels == 1, out_pixels)

Write out the final HAND tile

In [ ]:
from asf_tools.raster import write_cog

hand_raster = Path(dem_tile).name.replace('DEM.tif', f'HAND_ACC{accumulation_threshold}.tif')
write_cog(hand_raster, out_pixels, transform=dem.transform.to_gdal(), epsg_code=dem.crs.to_epsg())

Check out the HAND you made!

In [ ]:
import rasterio.plot

with rasterio.open(hand_raster) as hand:
    rasterio.plot.show(hand, title=hand_raster, vmin=0, vmax=10, cmap='binary')